In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import rasterio
from rasterio.transform import from_origin
import numpy as np
from rasterio.warp import calculate_default_transform, reproject, Resampling

In [2]:
def thermal_growingseason_vec(ta, tlim=5.0):
    doy = np.arange(1, len(ta) + 1)
    gs_start = np.nan
    gs_end = np.nan

    for d in range(6, len(ta)):
        if np.all(ta[d-6:d] >= tlim):
            gs_start = doy[d]
            break

    if np.isnan(gs_start):
        return np.nan, np.nan, np.nan

    for d in range(max(186, int(gs_start)), len(ta)):
        if np.all(ta[d-6:d] <= tlim):
            gs_end = doy[d]
            break

    if np.isnan(gs_end):
        gs_end = doy[-1]

    return int(gs_start), int(gs_end), int(gs_end - gs_start)

def add_growing_season_to_dataset(data, tlim=5.0):
    ta = data['Tavg']
    years = np.unique(ta['time.year'].values)
    lat, lon = data['lat'].values, data['lon'].values

    gs_mask = xr.full_like(ta, 0, dtype='int8')

    for year in years:
        print(f"Processing year {year}")
        ta_year = ta.sel(time=str(year))
        if ta_year.time.size < 200:
            continue
        ta_values = ta_year.transpose('lat', 'lon', 'time').values
        time_index = ta_year.time.dt.dayofyear.values

        for i in range(len(lat)):
            for j in range(len(lon)):
                ts = ta_values[i, j, :]
                if np.isnan(ts).all():
                    continue
                start, end, length = thermal_growingseason_vec(ts, tlim)
                if not np.isnan(start) and not np.isnan(end):
                    mask = (time_index >= start) & (time_index <= end)
                    gs_mask.loc[dict(lat=lat[i], lon=lon[j], time=ta_year.time[mask])] = 1

    data['growing_season_mask'] = gs_mask
    data['growing_season_mask'].attrs['description'] = '0 = outside growing season, 1 = during growing season'
    data['growing_season_mask'].attrs['based_on'] = 'Thermal definition, Linderholm et al. 2008, Tavg > 5°C'
    data['growing_season_mask'].attrs['units'] = 'binary'

    return data

In [3]:
file = r'/scratch/project_2000908/nousu/weather_data/data10x10_1980_2021_with_derivatives.nc'
data = xr.open_dataset(file)

In [4]:
data = add_growing_season_to_dataset(data)

Processing year 1980
Processing year 1981
Processing year 1982
Processing year 1983
Processing year 1984
Processing year 1985
Processing year 1986
Processing year 1987
Processing year 1988
Processing year 1989
Processing year 1990
Processing year 1991
Processing year 1992
Processing year 1993
Processing year 1994
Processing year 1995
Processing year 1996
Processing year 1997
Processing year 1998
Processing year 1999
Processing year 2000
Processing year 2001
Processing year 2002
Processing year 2003
Processing year 2004
Processing year 2005
Processing year 2006
Processing year 2007
Processing year 2008
Processing year 2009
Processing year 2010
Processing year 2011
Processing year 2012
Processing year 2013
Processing year 2014
Processing year 2015
Processing year 2016
Processing year 2017
Processing year 2018
Processing year 2019
Processing year 2020
Processing year 2021


In [5]:
# group by year
gs_by_year = data['growing_season_mask'].groupby('time.year')

# sum the growing season days per year
gs_length_per_year = gs_by_year.sum(dim='time', skipna=True)

# average growing season length across years
avg_gs_length = gs_length_per_year.mean(dim='year', skipna=True)

In [6]:
data_avg = xr.Dataset()
data_avg['air_temperature'] = data['Tavg'].mean(dim='time')
data_avg['vapor_pressure_deficit'] = data['vpd'].where(data['growing_season_mask'] == 1).mean(dim='time', skipna=True)
data_avg['growing_season_length'] = avg_gs_length
data_avg = data_avg.where(np.isfinite(data['Tavg'].mean(dim='time')))

In [7]:
# Extract data
for variable in ['air_temperature', 'vapor_pressure_deficit', 'growing_season_length']:

    output_file_kkj = f'/scratch/project_2000908/nousu/mNFI_to_NEP/METEO/avg_weather_{variable}_kkj.tif'
    output_file_tm35fin = f'/scratch/project_2000908/nousu/mNFI_to_NEP/METEO/{variable}.tif'

    datas = data_avg[variable]
    lat = datas['lat'].values
    lon = datas['lon'].values
    values = datas.values
    values = np.flipud(values)  # flip vertically
    
    nodata_value = -9999
    values[np.isnan(values)] = nodata_value
    
    res_y = np.abs(lat[1] - lat[0])
    res_x = np.abs(lon[1] - lon[0])
    
    # Original transform in KKJ
    transform_kkj = from_origin(west=lon.min() - res_x / 2,
                                north=lat.max() + res_y / 2,
                                xsize=res_x,
                                ysize=res_y)
    
    # Step 1: Save in original KKJ CRS (EPSG:2393 or your actual KKJ zone)
    with rasterio.open(
        output_file_kkj,
        "w",
        driver="GTiff",
        height=values.shape[0],
        width=values.shape[1],
        count=1,
        dtype=values.dtype,
        crs="EPSG:2393",  # KKJ / Finland zone 3
        transform=transform_kkj,
        nodata=nodata_value
    ) as dst:
        dst.write(values, 1)
    
    # Step 2: Reproject to EPSG:3067
    with rasterio.open(output_file_kkj) as src:
        transform_3067, width, height = calculate_default_transform(
            src.crs, "EPSG:3067", src.width, src.height, *src.bounds)
    
        kwargs = src.meta.copy()
        kwargs.update({
            "crs": "EPSG:3067",
            "transform": transform_3067,
            "width": width,
            "height": height
        })
    
        with rasterio.open(output_file_tm35fin, "w", **kwargs) as dst:
            reproject(
                source=rasterio.band(src, 1),
                destination=rasterio.band(dst, 1),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform_3067,
                dst_crs="EPSG:3067",
                resampling=Resampling.nearest,
                src_nodata=nodata_value,
                dst_nodata=nodata_value
            )
    
    print(f"Saved {variable} reference grid to EPSG:3067")

Saved air_temperature reference grid to EPSG:3067
Saved vapor_pressure_deficit reference grid to EPSG:3067
Saved growing_season_length reference grid to EPSG:3067
